In [ ]:
# major errors, please make it work for baseline again
# issue might be due to embeddings
# troubleshooting because the results don't make sense

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

# Setup

In [ ]:
x_train_full = np.load("../input/ndscloaded/x_train.npy")
y_train_full = np.load("../input/ndscloaded/y_train.npy")
x_test_full = np.load("../input/ndscloaded/x_test.npy")
features = np.load("../input/ndscloaded/features.npy")
test_features = np.load("../input/ndscloaded/test_features.npy")
word_index = np.load("../input/ndscloaded/word_index.npy").item()
embedding_matrix = np.load("../input/ndscloaded/embedding_matrix.npy")

In [ ]:
print(np.shape(x_train_full))
print(x_train_full)
print(np.shape(y_train_full))
print(y_train_full)
print(np.shape(x_test_full))
print(x_test_full)
print(np.shape(embedding_matrix))

In [ ]:
# batch_size = 128

# indices = np.arange(len(x_train_full))
# # np.random.shuffle(indices)
# i = 0
# x_train_batch_tokens = np.take(x_train_full, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
# x_train_batch = [[embedding_matrix[token] for token in title][::-1] for title in x_train_batch_tokens]
# # [::-1] because I want it back in the correct order
# y_train_batch_targets = np.take(y_train_full, indices[i*batch_size:(i+1)*batch_size])
# y_train_batch = np.zeros((batch_size, 58))
# y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

# print(indices)
# print(x_train_batch_tokens)
# print(np.shape(x_train_batch))
# print(y_train_batch_targets)
# print(y_train_batch)

# print(type(indices))
# print(type(x_train_batch_tokens))
# print(type(np.shape(x_train_batch[:1])))
# print(type(x_train_batch))
# print(type(y_train_batch_targets))
# print(type(y_train_batch))

In [ ]:
import pandas as cv
test_df = pd.read_csv("../input/ndsc-beginner/test.csv")
test_df.head()

# Validation split

In [ ]:
!ls ../input/ndscindexes/ndscindexes
!ls ../input/ndscindexes/ndscindexes/trn_index/
!ls ../input/ndscindexes/ndscindexes/trn_index/face/

In [ ]:
import numpy as np
import glob

# c for face
trn_index_np_c = glob.glob("../input/ndscindexes/ndscindexes/trn_index/face/*")  
val_index_np_c = glob.glob("../input/ndscindexes/ndscindexes/val_index/face/*") 

trn_index_c = np.concatenate([np.load(i) for i in trn_index_np_c]).ravel()
val_index_c = np.concatenate([np.load(i) for i in val_index_np_c]).ravel()

# l for lips
trn_index_np_l = glob.glob("../input/ndscindexes/ndscindexes/trn_index/lip/*")
val_index_np_l = glob.glob("../input/ndscindexes/ndscindexes/val_index/lip/*")

trn_index_l = np.concatenate([np.load(i) for i in trn_index_np_l]).ravel()
val_index_l = np.concatenate([np.load(i) for i in val_index_np_l]).ravel()

# b for beauty (combined) 
trn_index_b = np.concatenate([trn_index_c,trn_index_l]).ravel()
val_index_b = np.concatenate([val_index_c,val_index_l]).ravel()
tes_index_b = np.argwhere(test_df["image_path"].str[0] == "b").ravel()

# f for fashion
trn_index_np_f = glob.glob("../input/ndscindexes/ndscindexes/trn_index/fashion/*")
val_index_np_f = glob.glob("../input/ndscindexes/ndscindexes/val_index/fashion/*")

trn_index_f = np.concatenate([np.load(i) for i in trn_index_np_f]).ravel()
val_index_f = np.concatenate([np.load(i) for i in val_index_np_f]).ravel()
tes_index_f = np.argwhere(test_df["image_path"].str[0] == "f").ravel()

# m for mobile
trn_index_np_m = glob.glob("../input/ndscindexes/ndscindexes/trn_index/mobile/*")
val_index_np_m = glob.glob("../input/ndscindexes/ndscindexes/val_index/mobile/*")

trn_index_m = np.concatenate([np.load(i) for i in trn_index_np_m]).ravel()
val_index_m = np.concatenate([np.load(i) for i in val_index_np_m]).ravel()
tes_index_m = np.argwhere(test_df["image_path"].str[0] == "m").ravel()

In [ ]:
print(np.shape(trn_index_b))
print(np.shape(trn_index_f))
print(np.shape(trn_index_m))

# Training

In [ ]:
from keras.models import Model, clone_model
from keras.models import Sequential, Model
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Activation, Input, Dropout
from keras.layers import Permute, Reshape, concatenate, Flatten
from keras.layers import BatchNormalization
from keras.layers import Input, CuDNNLSTM, Dense, Bidirectional, Activation, Flatten, RepeatVector, Permute
from keras.layers import merge
from keras import regularizers

In [ ]:
# model = Sequential()
# model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
# #                         input_shape=(100, 300)
#                         input_shape=(100, 300)
#                        ))
# model.add(Bidirectional(CuDNNLSTM(64)))
# model.add(Dense(58))
# model.add(Activation('softmax'))

In [ ]:
INPUT_DIM = 300
TIME_STEPS = 100
# if True, the attention vector is shared across the input_dimensions where the attention is applied.
SINGLE_ATTENTION_VECTOR = False

def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = concatenate([inputs, a_probs])
    return output_attention_mul

inputs_embed = Input((TIME_STEPS, INPUT_DIM))

lstm_1 = CuDNNLSTM(64, return_sequences=True)(inputs_embed)
lstm_2 = CuDNNLSTM(64, return_sequences=True)(lstm_1)

attention = attention_3d_block(lstm_2)
attention = Flatten()(attention)

fc_layer = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(attention)
fc_layer = BatchNormalization()(fc_layer)
fc_layer = Dropout(0.2)(fc_layer)
outputs = Dense(58, activation='softmax')(fc_layer)

x_val = np.take(x_train_full, val_index_b, axis = 0)
y_val = np.take(y_train_full, val_index_b, axis = 0)

model = Model(inputs=inputs_embed, outputs=outputs)
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              validation_data=(x_val, y_val),
              metrics=['accuracy'])

In [ ]:
# Data providers
batch_size = 128

# print(len(x_train))  # note that x_train is no longer constant, same with y_train
# print(np.random.shuffle())

def batch_gen():
    indices = np.arange(len(x_train)-1)
    n_batches = math.floor(len(x_train) / batch_size)
    while True: 
        np.random.shuffle(indices)  # Shuffle the data.
        for i in range(n_batches):
            x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
            x_train_batch = [[embedding_matrix[token] for token in title][::-1] for title in x_train_batch_tokens]
            # [::-1] because I want it back in the correct order
            y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
#             y_train_batch = np.zeros((batch_size, 58))
#             y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1
            yield (np.array(x_train_batch), y_train_batch_targets)
            
mg = batch_gen()

# Training for model BEAUTY

In [ ]:
print(np.shape(x_val))
print(np.shape(y_val))

In [ ]:
import keras

x_train = np.take(x_train_full, trn_index_b, axis = 0)
y_train = np.take(y_train_full, trn_index_b, axis = 0)
print(np.shape(x_train))
print(np.shape(y_train))
x_val_tokens = np.take(x_train_full, val_index_b, axis = 0)[:10000]
x_val = np.array([[embedding_matrix[token] for token in title][::-1] for title in x_val_tokens])
y_val = np.take(y_train_full, val_index_b, axis = 0)[:10000]
print(np.shape(x_val))
print(np.shape(y_val))
model_beauty = keras.models.clone_model(model)
model_beauty.set_weights(model.get_weights())
model_beauty.compile(optimizer='adam', 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])
mg = batch_gen()

In [ ]:
model_beauty.fit_generator(mg, epochs=20,
                           steps_per_epoch=1000,
                           validation_data=(x_val, y_val),
                           verbose=True)

In [ ]:
model_beauty.fit_generator(mg, epochs=2,
                           steps_per_epoch=1000,
                           validation_data=(x_val, y_val),
                           verbose=True)

# Training for model FASHION

In [ ]:
x_train = np.take(x_train_full, trn_index_f, axis = 0)
y_train = np.take(y_train_full, trn_index_f, axis = 0)
print(np.shape(x_train))
print(np.shape(y_train))
x_val_tokens = np.take(x_train_full, val_index_f, axis = 0)[:10000]
x_val = np.array([[embedding_matrix[token] for token in title][::-1] for title in x_val_tokens])
y_val = np.take(y_train_full, val_index_f, axis = 0)[:10000]
print(np.shape(x_val))
print(np.shape(y_val))
model_fashion = keras.models.clone_model(model)
model_fashion.set_weights(model.get_weights())
model_fashion.compile(optimizer='adam', 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])
mg = batch_gen()

In [ ]:
model_fashion.fit_generator(mg, epochs=20,
                           steps_per_epoch=1000,
                           validation_data=(x_val, y_val),
                           verbose=True)

In [ ]:
model_fashion.fit_generator(mg, epochs=2,
                           steps_per_epoch=1000,
                           validation_data=(x_val, y_val),
                           verbose=True)

# Training for model MOBILE

In [ ]:
x_train = np.take(x_train_full, trn_index_m, axis = 0)
y_train = np.take(y_train_full, trn_index_m, axis = 0)
print(np.shape(x_train))
print(np.shape(y_train))
x_val_tokens = np.take(x_train_full, val_index_m, axis = 0)[:10000]
x_val = np.array([[embedding_matrix[token] for token in title][::-1] for title in x_val_tokens])
y_val = np.take(y_train_full, val_index_m, axis = 0)[:10000]
print(np.shape(x_val))
print(np.shape(y_val))
model_mobile = keras.models.clone_model(model)
model_mobile.set_weights(model.get_weights())
model_mobile.compile(optimizer='adam', 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])
mg = batch_gen()

In [ ]:
model_mobile.fit_generator(mg, epochs=20,
                           steps_per_epoch=1000,
                           validation_data=(x_val, y_val),
                           verbose=True)

In [ ]:
model_mobile.fit_generator(mg, epochs=2,
                           steps_per_epoch=1000,
                           validation_data=(x_val, y_val),
                           verbose=True)

# VALIDATION

In [ ]:
del x_val  # running of memory :(
del y_val

In [ ]:
import time
# prediction part
batch_size = 128
def batch_gen():
    n_batches = math.ceil(len(x_test) / batch_size)
    for i in range(n_batches):
        x_test_batch_tokens = x_test[i*batch_size:(i+1)*batch_size]
        x_test_batch = [[embedding_matrix[token] for token in title][::-1] for title in x_test_batch_tokens]
        # [::-1] because I want it back in the correct order
        yield np.array(x_test_batch)

In [ ]:
test_df = pd.read_csv("../input/ndsc-beginner/test.csv")
test_df["Supercategory"] = test_df["image_path"].str[0]
supercats = np.array(test_df["Supercategory"])
supercat_dict = {
    "b" : np.array([1]*17 + [0]*14 + [0]*27),
    "f" : np.array([0]*17 + [1]*14 + [0]*27),
    "m" : np.array([0]*17 + [0]*14 + [1]*27)
}

In [ ]:
x_test = np.take(x_train_full, val_index_b, axis = 0)
y_test = np.take(y_train_full, val_index_b, axis = 0)
preds_beauty_val = []
for x in batch_gen():
    preds_beauty_val.extend(model_beauty.predict(x))
correct_beauty_val = np.sum(np.equal(y_test, [np.argmax(pred) for pred in preds_beauty_val]))

In [ ]:
x_test = np.take(x_train_full, val_index_f, axis = 0)
y_test = np.take(y_train_full, val_index_f, axis = 0)
preds_fashion_val = []
for x in batch_gen():
    preds_fashion_val.extend(model_fashion.predict(x))
correct_fashion_val = np.sum(np.equal(y_test, [np.argmax(pred) for pred in preds_fashion_val]))

In [ ]:
x_test = np.take(x_train_full, val_index_m, axis = 0)
y_test = np.take(y_train_full, val_index_m, axis = 0)
preds_mobile_val = []
for x in batch_gen():
    preds_mobile_val.extend(model_mobile.predict(x))
correct_mobile_val = np.sum(np.equal(y_test, [np.argmax(pred) for pred in preds_mobile_val]))

In [ ]:
correct_total_val = correct_beauty_val + correct_fashion_val + correct_mobile_val
total_val_entries = len(preds_beauty_val) + len(preds_fashion_val) + len(preds_mobile_val)

print("beauty : ", correct_beauty_val, "/", len(preds_beauty_val), 
      "{:5f}".format(correct_beauty_val/len(preds_beauty_val)))
print("fashion : ", correct_fashion_val, "/", len(preds_fashion_val), 
      "{:5f}".format(correct_fashion_val/len(preds_fashion_val)))
print("mobile : ", correct_mobile_val, "/", len(preds_mobile_val), 
      "{:5f}".format(correct_mobile_val/len(preds_mobile_val)))
print("total : ", correct_total_val, "/", total_val_entries, 
      "{:5f}".format(correct_total_val/total_val_entries))

# INFERENCE

In [ ]:
x_test = np.take(x_test_full, tes_index_b, axis = 0)
preds_beauty = []
for x in batch_gen():
    preds_beauty.extend(model_beauty.predict(x))

In [ ]:
x_test = np.take(x_test_full, tes_index_f, axis = 0)
preds_fashion = []
for x in batch_gen():
    preds_fashion.extend(model_fashion.predict(x))

In [ ]:
x_test = np.take(x_test_full, tes_index_m, axis = 0)
preds_mobile = []
for x in batch_gen():
    preds_mobile.extend(model_mobile.predict(x))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(preds_beauty[0])
plt.plot(preds_fashion[0])
plt.plot(preds_mobile[0])
plt.yscale("log")
plt.show()

In [ ]:
all_preds = ["None"]*len(x_test_full)

for i,x in zip(tes_index_b,preds_beauty):
    all_preds[i] = x
for i,x in zip(tes_index_f,preds_fashion):
    all_preds[i] = x
for i,x in zip(tes_index_m,preds_mobile):
    all_preds[i] = x

all_preds = np.array(all_preds)
print(np.shape(all_preds))

In [ ]:
print(np.shape(all_preds))
y_te = [np.argmax(pred) for pred,supercat in zip(all_preds,supercats)]

submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission_no_cheat.csv", index=False)

In [ ]:
print(np.shape(all_preds))
y_te = [np.argmax(pred*supercat_dict[supercat]) for pred,supercat in zip(all_preds,supercats)]

submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission.csv", index=False)

In [ ]:
submit_df.head()

In [ ]:
submit_df.tail()